In [1]:
%load_ext autoreload

In [39]:
%autoreload 2
import math
import numpy as np

from BoseHubbardHamiltonian import BoseHubbardHamiltonian

from qiskit.quantum_info import Pauli, Operator
from qiskit.quantum_info import SparsePauliOp

from symmer.operators import PauliwordOp, QuantumState
from symmer.projection import QubitTapering 

import rustworkx as rx
from qiskit_nature.second_q.hamiltonians.lattices import (
    BoundaryCondition,
    HyperCubicLattice,
    Lattice,
    LatticeDrawStyle,
    LineLattice,
    SquareLattice,
    TriangularLattice,
)

from symmer.operators import IndependentOp
from scipy.linalg import eigh
import scipy.sparse

In [46]:
#line lattice 
num_nodes = 2 #number of sites in the lattice 
boundary_condition = BoundaryCondition.OPEN #open lattice
line_lattice = LineLattice(num_nodes=num_nodes, boundary_condition=boundary_condition)

In [45]:
#triangular lattice 
rows = 4
cols = 3
boundary_condition = BoundaryCondition.PERIODIC

triangular_lattice = TriangularLattice(rows=rows, cols=cols, boundary_condition=boundary_condition)

In [94]:

graph = rx.PyGraph(multigraph=False)  # multigraph shoud be False
graph.add_nodes_from(range(9))
weighted_edge_list = [
    (0, 5, 1.0),
    (0, 8, 1.0),
    (0, 1, 1.0), 
    (1, 2, 1.0), 
    (2, 3, 1.0), 
    (3, 4, 1.0),
    (5, 6, 1.0),
    (5, 4, 1.0),
    (2, 6, 1.0),
    (2, 7, 1.0),
    (7, 8, 1.0),
    (8, 4, 1.0),
    (5, 2, 1.0),
]
graph.add_edges_from(weighted_edge_list)

# make a lattice
general_lattice = Lattice(graph)
set(general_lattice.graph.weighted_edge_list())

{(0, 1, 1.0),
 (0, 5, 1.0),
 (0, 8, 1.0),
 (1, 2, 1.0),
 (2, 3, 1.0),
 (2, 6, 1.0),
 (2, 7, 1.0),
 (3, 4, 1.0),
 (5, 2, 1.0),
 (5, 4, 1.0),
 (5, 6, 1.0),
 (7, 8, 1.0),
 (8, 4, 1.0)}

In [95]:
#Hamiltonian of the system given the number of sites in the lattice and the number of qubits 
H_instance = BoseHubbardHamiltonian(9, 2, general_lattice)
H_q = H_instance.get_H(1,1)

In [96]:
#Hamiltonian to be tapered
H = PauliwordOp.from_qiskit(H_q)
print(H)

 0.933+0.000j IXIXIIIIIIIIIIIIII +
 0.483+0.000j IXXXIIIIIIIIIIIIII +
 0.483+0.000j IXYYIIIIIIIIIIIIII +
-0.250+0.000j IXZXIIIIIIIIIIIIII +
 0.933+0.000j IYIYIIIIIIIIIIIIII +
-0.483+0.000j IYXYIIIIIIIIIIIIII +
 0.483+0.000j IYYXIIIIIIIIIIIIII +
-0.250+0.000j IYZYIIIIIIIIIIIIII +
 0.483+0.000j XXIXIIIIIIIIIIIIII +
 0.250+0.000j XXXXIIIIIIIIIIIIII +
 0.250+0.000j XXYYIIIIIIIIIIIIII +
-0.129+0.000j XXZXIIIIIIIIIIIIII +
-0.483+0.000j XYIYIIIIIIIIIIIIII +
 0.250+0.000j XYXYIIIIIIIIIIIIII +
-0.250+0.000j XYYXIIIIIIIIIIIIII +
 0.129+0.000j XYZYIIIIIIIIIIIIII +
 0.483+0.000j YXIYIIIIIIIIIIIIII +
-0.250+0.000j YXXYIIIIIIIIIIIIII +
 0.250+0.000j YXYXIIIIIIIIIIIIII +
-0.129+0.000j YXZYIIIIIIIIIIIIII +
 0.483+0.000j YYIXIIIIIIIIIIIIII +
 0.250+0.000j YYXXIIIIIIIIIIIIII +
 0.250+0.000j YYYYIIIIIIIIIIIIII +
-0.129+0.000j YYZXIIIIIIIIIIIIII +
-0.250+0.000j ZXIXIIIIIIIIIIIIII +
-0.129+0.000j ZXXXIIIIIIIIIIIIII +
-0.129+0.000j ZXYYIIIIIIIIIIIIII +
 0.067+0.000j ZXZXIIIIIIIIIIIIII +
-0.250+0.000j ZYIYII

In [97]:
#IndipendentOp represents algebraically independent sets of Pauli operators for stabilizer manipulation/projections
IndependentOp.symmetry_generators(H, commuting_override=True)

1 IZIZIZIZIZIZIZIZIZ

In [81]:
taper_hamiltonian = QubitTapering(H)
taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis()

-1 IZIIIIIIIIIIIIII

In [82]:
print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand may be rotated onto the single-qubit Pauli operators\n')
print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
print('\nvia a sequence of rotations e^{i pi/4 R} where\n')
for index, (rot, angle) in enumerate(taper_hamiltonian.stabilizers.stabilizer_rotations):
    print(f'R_{index} = {rot}')

We are able to taper 1 qubits from the Hamiltonian.

The symmetry generators are

1 IZIZIZIZIZIZIZIZ

and may be rotated onto the single-qubit Pauli operators

-1 IZIIIIIIIIIIIIII

via a sequence of rotations e^{i pi/4 R} where

R_0 =  1.000+0.000j IYIZIZIZIZIZIZIZ
R_1 =  1.000+0.000j IYIIIIIIIIIIIIII


In [83]:
sec_array = [1]
ham_tap_p = taper_hamiltonian.taper_it(sector=sec_array)
print('Tapered Hamiltonian:\n')
print(ham_tap_p) #sector = +1

Tapered Hamiltonian:

 16.000+0.000j IIIIIIIIIIIIIII +
 1.000+0.000j IIIIIIIIIIIIIIZ +
-1.000+0.000j IIIIIIIIIIIIIZI +
-2.000+0.000j IIIIIIIIIIIIIZZ +
 1.000+0.000j IIIIIIIIIIIIZII +
-1.000+0.000j IIIIIIIIIIIZIII +
-2.000+0.000j IIIIIIIIIIIZZII +
 1.000+0.000j IIIIIIIIIIZIIII +
-1.000+0.000j IIIIIIIIIZIIIII +
-2.000+0.000j IIIIIIIIIZZIIII +
 1.000+0.000j IIIIIIIIZIIIIII +
-1.000+0.000j IIIIIIIZIIIIIII +
-2.000+0.000j IIIIIIIZZIIIIII +
 1.000+0.000j IIIIIIZIIIIIIII +
-1.000+0.000j IIIIIZIIIIIIIII +
-2.000+0.000j IIIIIZZIIIIIIII +
 1.000+0.000j IIIIZIIIIIIIIII +
-1.000+0.000j IIIZIIIIIIIIIII +
-2.000+0.000j IIIZZIIIIIIIIII +
 1.000+0.000j IIZIIIIIIIIIIII +
 1.000-0.000j IIZIZIZIZIZIZIZ +
-1.000+0.000j IZIIIIIIIIIIIII +
-2.000+0.000j IZZIIIIIIIIIIII +
-1.000+0.000j ZIIIIIIIIIIIIII +
-2.000+0.000j ZIZIZIZIZIZIZIZ +
-0.933+0.000j IIIIIIIIIIIIIIX +
 0.250-0.000j IIIIIIIIIIIIIZX +
 0.933+0.000j IIZIZIZIZIZIZIX +
-0.250+0.000j IIZIZIZIZIZIZZX +
 0.250-0.000j ZIIIIIIIIIIIIIX +
-0.067+0.000j ZII

In [12]:
sec_array = [-1]
ham_tap_n = taper_hamiltonian.taper_it(sector=sec_array)
print('Tapered Hamiltonian:\n')
print(ham_tap_n) #sector = -1

Tapered Hamiltonian:

 4.000+0.000j III +
-1.000+0.000j IZI +
-2.000+0.000j IZZ +
-1.000+0.000j ZII +
 2.000+0.000j ZIZ +
 1.866+0.000j IIX +
-0.500+0.000j IZX +
-0.500+0.000j ZIX +
 0.134+0.000j ZZX +
 0.500+0.000j XXX +
 0.500+0.000j XYY +
-0.500+0.000j YXY +
 0.500+0.000j YYX


In [14]:
H_sparse_matrix = H.to_sparse_matrix
H_array = H_sparse_matrix.toarray()

In [24]:
np.set_printoptions(precision=20)
eigh(H_array, eigvals_only=True)

array([-4.6410161513775117e-01, -2.9150262212917688e-01,
        1.5543122344752192e-15,  1.7763568394002505e-15,
        9.1723746970178299e-01,  1.0000000000000022e+00,
        2.0000000000000018e+00,  2.7639320225002111e+00,
        3.0000000000000004e+00,  3.0000000000000036e+00,
        6.0000000000000018e+00,  6.4641016151377562e+00,
        7.2360679774997898e+00,  9.0000000000000000e+00,
        1.0291502622129187e+01,  1.3082762530298218e+01])

In [23]:
ham_tap_p_sparse_matrix = ham_tap_p.to_sparse_matrix
ham_tap_p_array = ham_tap_p_sparse_matrix.toarray()
eigenvalues_p = eigh(ham_tap_p_array, eigvals_only=True)
np.set_printoptions(precision=20)
print(eigenvalues_p)

[8.8817841970012523e-16 1.8358983851527811e-15 9.1723746970178111e-01
 2.0000000000000009e+00 2.7639320225002115e+00 6.0000000000000036e+00
 7.2360679774997863e+00 1.3082762530298224e+01]


In [22]:
ham_tap_n_sparse_matrix = ham_tap_n.to_sparse_matrix
ham_tap_n_array = ham_tap_n_sparse_matrix.toarray()
eigenvalues_n = eigh(ham_tap_n_array, eigvals_only=True)
np.set_printoptions(precision=20)
print(eigenvalues_n)

[-0.4641016151377542 -0.2915026221291805  1.0000000000000013
  3.0000000000000018  3.000000000000002   6.464101615137756
  9.000000000000002  10.291502622129185 ]
